In [11]:
!pip install ogb
!pip install torch torchvision torchaudio
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric


from ogb.nodeproppred import PygNodePropPredDataset

dataset = PygNodePropPredDataset(name = "ogbn-arxiv") 

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
graph = dataset[0] # pyg graph object


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ImportError: cannot import name 'PygNodePropPredDataset' from 'ogb.nodeproppred' (/home/kriti-arora/.local/lib/python3.8/site-packages/ogb/nodeproppred/__init__.py)

In [ ]:
# Check the number of nodes and edges
num_nodes = graph.num_nodes
num_edges = graph.num_edges
print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")
